In [1]:
import pandas as pd
import os
import time

In [2]:
import joblib
from joblib import Parallel,delayed

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
stock_basic = pd.read_csv("stock_basic.csv")
stock_basic

,ts_code,symbol,name,area,industry,list_date
0,000001.SZ,1,平安银行,深圳,银行,19910403
1,000002.SZ,2,万科A,深圳,全国地产,19910129
2,000004.SZ,4,国华网安,深圳,软件服务,19910114
3,000006.SZ,6,深振业A,深圳,区域地产,19920427
4,000007.SZ,7,全新好,深圳,其他商业,19920413
...,...,...,...,...,...,...
4530,605580.SH,605580,恒盛能源,浙江,供气供热,20210819
4531,605588.SH,605588,冠石科技,江苏,元器件,20210812
4532,605589.SH,605589,圣泉集团,山东,化工原料,20210810
4533,605598.SH,605598,上海港湾,上海,建筑工程,20210917


In [5]:
trade_cal=pd.read_csv("trade_cal.csv")
trade_cal

,exchange,cal_date,is_open,pretrade_date
0,SSE,20141027,1,20141024
1,SSE,20141028,1,20141027
2,SSE,20141029,1,20141028
3,SSE,20141030,1,20141029
4,SSE,20141031,1,20141030
...,...,...,...,...
2427,SSE,20241021,1,20241018
2428,SSE,20241022,1,20241021
2429,SSE,20241023,1,20241022
2430,SSE,20241024,1,20241023


In [6]:
custom_date = pd.read_csv("custom_date.csv")
start_date=str(custom_date.start_date[0])
end_date=str(custom_date.end_date[0])
print(start_date)
print(end_date)

20141027
20241027


In [7]:
swl2 = pd.read_csv("swl2.csv")
swl2

,index_code,industry_name,level,industry_code,is_pub,parent_code,src
0,801016.SI,种植业,L2,110100,1,110000,SW2021
1,801015.SI,渔业,L2,110200,1,110000,SW2021
2,801011.SI,林业Ⅱ,L2,110300,0,110000,SW2021
3,801014.SI,饲料,L2,110400,1,110000,SW2021
4,801012.SI,农产品加工,L2,110500,1,110000,SW2021
...,...,...,...,...,...,...,...
129,801971.SI,环境治理,L2,760100,1,760000,SW2021
130,801972.SI,环保设备Ⅱ,L2,760200,1,760000,SW2021
131,801981.SI,个护用品,L2,770100,1,770000,SW2021
132,801982.SI,化妆品,L2,770200,1,770000,SW2021


In [8]:
# swl2.industry_code

In [9]:
# stock_basic.ts_code

In [10]:
all_index = stock_basic.shape[0]
df_index = pd.to_datetime(trade_cal.cal_date.tolist(),format='%Y%m%d')

In [12]:
path = "stock_pool/industry"
start_date=int(start_date)
path1 = "stock_pool/industry_1"
@joblib.delayed
def task(stock_name):
    current_df=pd.read_csv(path+"/"+stock_name+".csv")
    industry = pd.DataFrame(index=trade_cal.cal_date.tolist(),columns=swl2.index_code.tolist())
    industry["ts_code"]=stock_name
    industry["trade_date"]=industry.index.tolist()
    loop_upper = current_df.shape[0]
    for i in range(0, loop_upper):
        row = current_df.iloc[i]
        in_date = row.in_date
        l2_code = row.l2_code
        if in_date<start_date:
            in_date=start_date
        if i<loop_upper-1 :
            next_row = current_df.iloc[i+1]
            end_date = next_row.in_date
#             print(in_date,end_date)
            industry[l2_code].loc[in_date:]=1
            industry[l2_code].loc[end_date:]=0
        else:
            industry[l2_code].loc[in_date:]=1
    industry.to_csv(path1+"/"+stock_name+".csv",index=0)
#     return industry

tasks=[]

for index, row in stock_basic.iterrows():
    stock_name = stock_basic.ts_code[index]
    tasks.append(task(stock_name))
    
# df = pd.concat(
joblib.Parallel(n_jobs=5, verbose=10)(tasks)
#     ), axis=0)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   3 tasks      | elapsed:    1.4s
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    1.5s
[Parallel(n_jobs=5)]: Done  15 tasks      | elapsed:    1.6s
[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed:    1.7s
[Parallel(n_jobs=5)]: Batch computation too fast (0.1952s.) Setting batch_size=2.
[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed:    1.9s
[Parallel(n_jobs=5)]: Done  45 tasks      | elapsed:    2.1s
[Parallel(n_jobs=5)]: Done  67 tasks      | elapsed:    2.6s
[Parallel(n_jobs=5)]: Done  89 tasks      | elapsed:    2.9s
[Parallel(n_jobs=5)]: Done 115 tasks      | elapsed:    3.3s
[Parallel(n_jobs=5)]: Done 141 tasks      | elapsed:    3.7s
[Parallel(n_jobs=5)]: Done 171 tasks      | elapsed:    4.2s
[Parallel(n_jobs=5)]: Done 201 tasks      | elapsed:    4.7s
[Parallel(n_jobs=5)]: Done 235 tasks      | elapsed:    5.3s
[Parallel(n_jobs=5)]: Done 269 tasks      | elapse

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [14]:
@joblib.delayed
def task(stock_name):
    current_df=pd.read_csv(path1+"/"+stock_name+".csv")
    return current_df

tasks=[]

for index, row in stock_basic.iterrows():
    stock_name = stock_basic.ts_code[index]
    tasks.append(task(stock_name))
    
df = pd.concat(
    joblib.Parallel(n_jobs=5, verbose=10)(tasks
    ), axis=0)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   3 tasks      | elapsed:    1.2s
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    1.3s
[Parallel(n_jobs=5)]: Done  15 tasks      | elapsed:    1.3s
[Parallel(n_jobs=5)]: Batch computation too fast (0.1961s.) Setting batch_size=2.
[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed:    1.4s
[Parallel(n_jobs=5)]: Done  32 tasks      | elapsed:    1.5s
[Parallel(n_jobs=5)]: Batch computation too fast (0.1135s.) Setting batch_size=4.
[Parallel(n_jobs=5)]: Done  50 tasks      | elapsed:    1.6s
[Parallel(n_jobs=5)]: Done  94 tasks      | elapsed:    2.0s
[Parallel(n_jobs=5)]: Done 138 tasks      | elapsed:    2.5s
[Parallel(n_jobs=5)]: Done 190 tasks      | elapsed:    2.9s
[Parallel(n_jobs=5)]: Done 242 tasks      | elapsed:    3.5s
[Parallel(n_jobs=5)]: Done 302 tasks      | elapsed:    4.0s
[Parallel(n_jobs=5)]: Done 362 tasks      | elapsed:    4.5s
[Parallel(n_jobs=5)]: Done 43

In [15]:
# df

,801016.SI,801015.SI,801011.SI,801014.SI,801012.SI,801017.SI,801018.SI,801019.SI,801033.SI,801034.SI,...,801961.SI,801962.SI,801963.SI,801971.SI,801972.SI,801981.SI,801982.SI,801983.SI,ts_code,trade_date
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000001.SZ,20141027
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000001.SZ,20141028
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000001.SZ,20141029
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000001.SZ,20141030
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000001.SZ,20141031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,605599.SH,20241021
2428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,605599.SH,20241022
2429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,605599.SH,20241023
2430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,605599.SH,20241024


In [20]:
df1 = df.set_index(['trade_date']).sort_index()
df1.fillna(0, inplace=True)
df1

,801016.SI,801015.SI,801011.SI,801014.SI,801012.SI,801017.SI,801018.SI,801019.SI,801033.SI,801034.SI,...,801952.SI,801961.SI,801962.SI,801963.SI,801971.SI,801972.SI,801981.SI,801982.SI,801983.SI,ts_code
trade_date,,,,,,,,,,,,,,,,,,,,,
20141027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000001.SZ
20141027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,002261.SZ
20141027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,601801.SH
20141027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,002259.SZ
20141027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,601808.SH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20241025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600216.SH
20241025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,300046.SZ
20241025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,603170.SH


In [21]:
df1.loc[20141027]

,801016.SI,801015.SI,801011.SI,801014.SI,801012.SI,801017.SI,801018.SI,801019.SI,801033.SI,801034.SI,...,801952.SI,801961.SI,801962.SI,801963.SI,801971.SI,801972.SI,801981.SI,801982.SI,801983.SI,ts_code
trade_date,,,,,,,,,,,,,,,,,,,,,
20141027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000001.SZ
20141027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,002261.SZ
20141027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,601801.SH
20141027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,002259.SZ
20141027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,601808.SH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20141027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600233.SH
20141027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,605598.SH
20141027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,300833.SZ


In [32]:
path2 = "industry"
for index, row in trade_cal.iterrows():
    print(index, trade_cal.shape[0])
    current_date = row.cal_date
    current_df = df1.loc[current_date]
    current_df.sort_values(by="ts_code",inplace=True)
#     break
    current_df.to_csv(path2+"/"+str(current_date)+".csv",index=0)
# current_df

0 2432
1 2432
2 2432
3 2432
4 2432
5 2432
6 2432
7 2432
8 2432
9 2432
10 2432
11 2432
12 2432
13 2432
14 2432
15 2432
16 2432
17 2432
18 2432
19 2432
20 2432
21 2432
22 2432
23 2432
24 2432
25 2432
26 2432
27 2432
28 2432
29 2432
30 2432
31 2432
32 2432
33 2432
34 2432
35 2432
36 2432
37 2432
38 2432
39 2432
40 2432
41 2432
42 2432
43 2432
44 2432
45 2432
46 2432
47 2432
48 2432
49 2432
50 2432
51 2432
52 2432
53 2432
54 2432
55 2432
56 2432
57 2432
58 2432
59 2432
60 2432
61 2432
62 2432
63 2432
64 2432
65 2432
66 2432
67 2432
68 2432
69 2432
70 2432
71 2432
72 2432
73 2432
74 2432
75 2432
76 2432
77 2432
78 2432
79 2432
80 2432
81 2432
82 2432
83 2432
84 2432
85 2432
86 2432
87 2432
88 2432
89 2432
90 2432
91 2432
92 2432
93 2432
94 2432
95 2432
96 2432
97 2432
98 2432
99 2432
100 2432
101 2432
102 2432
103 2432
104 2432
105 2432
106 2432
107 2432
108 2432
109 2432
110 2432
111 2432
112 2432
113 2432
114 2432
115 2432
116 2432
117 2432
118 2432
119 2432
120 2432
121 2432
122 2432
123

923 2432
924 2432
925 2432
926 2432
927 2432
928 2432
929 2432
930 2432
931 2432
932 2432
933 2432
934 2432
935 2432
936 2432
937 2432
938 2432
939 2432
940 2432
941 2432
942 2432
943 2432
944 2432
945 2432
946 2432
947 2432
948 2432
949 2432
950 2432
951 2432
952 2432
953 2432
954 2432
955 2432
956 2432
957 2432
958 2432
959 2432
960 2432
961 2432
962 2432
963 2432
964 2432
965 2432
966 2432
967 2432
968 2432
969 2432
970 2432
971 2432
972 2432
973 2432
974 2432
975 2432
976 2432
977 2432
978 2432
979 2432
980 2432
981 2432
982 2432
983 2432
984 2432
985 2432
986 2432
987 2432
988 2432
989 2432
990 2432
991 2432
992 2432
993 2432
994 2432
995 2432
996 2432
997 2432
998 2432
999 2432
1000 2432
1001 2432
1002 2432
1003 2432
1004 2432
1005 2432
1006 2432
1007 2432
1008 2432
1009 2432
1010 2432
1011 2432
1012 2432
1013 2432
1014 2432
1015 2432
1016 2432
1017 2432
1018 2432
1019 2432
1020 2432
1021 2432
1022 2432
1023 2432
1024 2432
1025 2432
1026 2432
1027 2432
1028 2432
1029 2432
1030 24

1750 2432
1751 2432
1752 2432
1753 2432
1754 2432
1755 2432
1756 2432
1757 2432
1758 2432
1759 2432
1760 2432
1761 2432
1762 2432
1763 2432
1764 2432
1765 2432
1766 2432
1767 2432
1768 2432
1769 2432
1770 2432
1771 2432
1772 2432
1773 2432
1774 2432
1775 2432
1776 2432
1777 2432
1778 2432
1779 2432
1780 2432
1781 2432
1782 2432
1783 2432
1784 2432
1785 2432
1786 2432
1787 2432
1788 2432
1789 2432
1790 2432
1791 2432
1792 2432
1793 2432
1794 2432
1795 2432
1796 2432
1797 2432
1798 2432
1799 2432
1800 2432
1801 2432
1802 2432
1803 2432
1804 2432
1805 2432
1806 2432
1807 2432
1808 2432
1809 2432
1810 2432
1811 2432
1812 2432
1813 2432
1814 2432
1815 2432
1816 2432
1817 2432
1818 2432
1819 2432
1820 2432
1821 2432
1822 2432
1823 2432
1824 2432
1825 2432
1826 2432
1827 2432
1828 2432
1829 2432
1830 2432
1831 2432
1832 2432
1833 2432
1834 2432
1835 2432
1836 2432
1837 2432
1838 2432
1839 2432
1840 2432
1841 2432
1842 2432
1843 2432
1844 2432
1845 2432
1846 2432
1847 2432
1848 2432
1849 2432
